In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pandapower as pp
from pandapower.networks import case1354pegase

In [ ]:
# Run the case file and check if there is any error
net = case1354pegase()

# Set the sgen NOT in service
net.sgen.in_service = False

# Set the maximum and minimum power output of the generator
net.gen.max_p_mw = 100
net.gen.min_p_mw = 0

# maximum line and transformer loading percent determine the convergence
# Set maximum line loading percentage
net.line.max_loading_percent = 10000

# Set transformer loading percent as unlimited
net.trafo.max_loading_percent = 1e20

# Set min_p_mw at slack bus
net.ext_grid.min_p_mw = 0 

# Set the sgen NOT in service
net.sgen.in_service = False

# Set maximum generation capacity
net.gen.max_p_mw = 500

# maximum line and transformer loading percent determine the convergence
# Set maximum line loading percentage
net.line.max_loading_percent = 400
# Set transformer loading percent as unlimited
net.trafo.max_loading_percent = 1e20

# Set min_p_mw at slack bus
net.ext_grid.min_p_mw = 0

pp.rundcopp(net)

In [ ]:
pp.to_excel(net, 'network/case1354_original_network.xlsx')

In [ ]:
# Change bus name from 'Name' to 'Bus_no'
net.bus.name.name = 'Bus_no'
# Bus no. starts from 0
net_index = net.bus.name.astype(int) - 1

In [ ]:
# Create DataFrame to store network data
network = pd.DataFrame()

# Set network index
network.index = net_index

In [ ]:
############################################################################################################

# trouble shooting issue with mismatched buses between network and poly_cost_index

############################################################################################################



# Read poly_cost_index
poly_cost_index = pd.read_csv('./network/poly_cost_index.csv', index_col=0, header=0)

# Assuming net.poly_cost is already loaded
# Filter net.poly_cost to have exactly 54 rows
net.poly_cost = net.poly_cost.iloc[:54]

# Check the shapes
print("poly_cost_index shape:", poly_cost_index.shape)
print("Adjusted net.poly_cost shape:", net.poly_cost.shape)

# Reshape poly_cost_index and check the length
reshaped_index = poly_cost_index.values.reshape((-1))
print("Reshaped index length:", len(reshaped_index))

# Ensure lengths match before assignment
if len(reshaped_index) == net.poly_cost.shape[0]:
    net.poly_cost.index = reshaped_index
    print("Index assignment successful.")
else:
    print(f"Length mismatch: Expected axis has {net.poly_cost.shape[0]} elements, new values have {len(reshaped_index)} elements")

print(poly_cost_index.shape)
print(net.poly_cost.shape)


In [ ]:
# Get features from 'bus
network['vn_kv'] = net.bus.vn_kv.values

# Get features from 'load'
net.load.index = net.load.bus
network['load_p_mw'] = net.load.p_mw

# Get features from 'gen'
net.gen.index = net.gen.bus
network['max_gen_p_mw'] = net.gen.max_p_mw
network['min_gen_p_mw'] = net.gen.min_p_mw

# Get features from 'ext_grid' (ext_grid bus no: 68)
net.ext_grid.index = net.ext_grid.bus
network['max_gen_p_mw'][net.ext_grid.index.values] = net.ext_grid.max_p_mw
network['min_gen_p_mw'][net.ext_grid.index.values] = net.ext_grid.min_p_mw

# Get features from 'poly_cost'
# This needs manual work
poly_cost_index = pd.read_csv('./network/poly_cost_index.csv', index_col=0, header=0)
net.poly_cost.index = poly_cost_index.values.reshape((-1))
network['price_1'] = net.poly_cost.cp1_eur_per_mw
network['price_2'] = net.poly_cost.cp2_eur_per_mw2

# Additional features

network['x_pos'] = net.bus_geodata.x
network['y_pos'] = net.bus_geodata.y

In [ ]:
branch = net.line[['from_bus', 'to_bus']]
branch

In [ ]:
## line values are off c_nf


branch_attr = net.line[['c_nf_per_km', 'r_ohm_per_km', 'x_ohm_per_km']]
branch_attr

In [ ]:

trafo = net.trafo[['hv_bus', 'lv_bus']]
trafo

In [ ]:
trafo_attr = net.trafo[['sn_mva', 'tap_neutral', 'tap_pos', 'tap_step_percent', 'vk_percent']]
trafo_attr

In [ ]:
network.to_csv('network/node_features.csv')
branch.to_csv('network/branch_index.csv')
branch_attr.to_csv('network/branch_attr.csv')
trafo.to_csv('network/trafo_index.csv')
trafo_attr.to_csv('network/trafo_attr.csv')

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import joblib

# Normalize branch atrributes
branch_attr_scaler = MinMaxScaler()
branch_attr_scaled = branch_attr_scaler.fit_transform(branch_attr)
pd.DataFrame(branch_attr_scaled).to_csv('network/branch_attr_scaled.csv')
joblib.dump(branch_attr_scaler, 'network/branch_attr_scaler.scaler')

# Normalize transformer attributes
trafo_attr_scaler = MinMaxScaler()
trafo_attr_scaled = trafo_attr_scaler.fit_transform(trafo_attr)
pd.DataFrame(trafo_attr_scaled).to_csv('network/trafo_attr_scaled.csv')
joblib.dump(trafo_attr_scaler, 'network/trafo_attr_scaler.scaler')